In [5]:
import numpy as np
import pandas as pd
import time
import datetime


In [6]:
data = pd.read_csv('worldweatheronline.com/result_data.csv')

In [7]:
table0 = data[['date', 'time', 'tempC', 'weatherDesc']]

In [8]:
table0.head()

,date,time,tempC,weatherDesc
0,2009-04-01,0,0,Light sleet
1,2009-04-01,1200,2,Light drizzle
2,2009-04-02,0,0,Heavy snow
3,2009-04-02,1200,1,Heavy snow
4,2009-04-03,0,-3,Moderate snow


In [9]:
table0['day'] = [time.strptime(n, "%Y-%m-%d").tm_mday for n in table0['date']]
table0['month'] = [time.strptime(n, "%Y-%m-%d").tm_mon for n in table0['date']]
table0['year'] = [time.strptime(n, "%Y-%m-%d").tm_year for n in table0['date']]
table0['status'] = [0 if n == 0 else 1 for n in table0['time']]
table0.drop(['date'], axis='columns', inplace=True)

/var/folders/yc/jct8g08s4pdfb9kr_t5r7m8r0000gq/T/ipykernel_20056/99024874.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  table0['day'] = [time.strptime(n, "%Y-%m-%d").tm_mday for n in table0['date']]
/var/folders/yc/jct8g08s4pdfb9kr_t5r7m8r0000gq/T/ipykernel_20056/99024874.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  table0['month'] = [time.strptime(n, "%Y-%m-%d").tm_mon for n in table0['date']]
/var/folders/yc/jct8g08s4pdfb9kr_t5r7m8r0000gq/T/ipykernel_20056/99024874.py:3: SettingWithCopyWarnin

In [10]:
table1 = table0.loc[(table0['month'] == 1)]
table2 = table0.loc[(table0['month'] == 2)]
table3 = table0.loc[(table0['month'] == 3)]
table4 = table0.loc[(table0['month'] == 4)]
table5 = table0.loc[(table0['month'] == 5)]
table6 = table0.loc[(table0['month'] == 6)]
table7 = table0.loc[(table0['month'] == 7)]
table8 = table0.loc[(table0['month'] == 8)]
table9 = table0.loc[(table0['month'] == 9)]
table10 = table0.loc[(table0['month'] == 10)]
table11 = table0.loc[(table0['month'] == 11)]
table12 = table0.loc[(table0['month'] == 12)]
table_np1 = table1.values
table_np2 = table2.values
table_np3 = table3.values
table_np4 = table4.values
table_np5 = table5.values
table_np6 = table6.values
table_np7 = table7.values
table_np8 = table8.values
table_np9 = table9.values
table_np10 = table10.values
table_np11 = table11.values
table_np12 = table12.values

In [11]:
table1.head()

,time,tempC,weatherDesc,day,month,year,status
550,0,-11,Light snow,1,1,2010,0
551,1200,-7,Freezing fog,1,1,2010,1
552,0,-13,Freezing fog,2,1,2010,0
553,1200,-9,Overcast,2,1,2010,1
554,0,-13,Moderate snow,3,1,2010,0


In [12]:
def preprocessing(data):
    processing_data = data
    for i in processing_data:
        if i[2] in {'Thundery outbreaks possible','Moderate rain',
                    'Light drizzle','Light freezing rain','Moderate or heavy rain shower',
                    'Patchy rain possible','Light rain shower','Patchy light drizzle',
                    'Light rain','Moderate or heavy rain with thunder',
                    'Moderate rain at times', 'Heavy rain','Patchy light rain',
                    'Patchy light rain with thunder'}:
            i[2] = 2
        elif i[2] in {'Light snow','Patchy moderate snow','Heavy freezing drizzle',
                      'Moderate snow', 'Heavy snow', 'Light snow showers',
                      'Moderate or heavy sleet', 'Moderate or heavy snow showers',
                      'Light sleet','Light sleet showers','Patchy snow possible',
                      'Blizzard','Patchy heavy snow','Patchy light snow'}:
            i[2] = 3
        elif i[2] in {'Fog','Overcast','Freezing fog','Mist','Partly cloudy'}:
            i[2] = 1
        else:
            i[2] = 0
    return processing_data

In [13]:
def func_coef(data):
    #total = data.shape[0]
    rainy = 0
    sunny = 0
    snowy = 0
    cloudy = 0
    matrix = np.zeros((4, 4))
    i = 0
    while i < data.shape[0]-1:
        if data[i][2] == 2:
            rainy += 1
            matrix[2][data[i+1][2]] += 1 
        elif data[i][2] == 3:
            snowy += 1
            matrix[3][data[i+1][2]] += 1
        elif data[i][2] == 1:
            cloudy += 1
            matrix[1][data[i+1][2]] += 1
        else:
            sunny += 1
            matrix[0][data[i+1][2]] += 1
        i += 1 
    if sunny > 0:
        matrix[0] = matrix[0] / sunny
    elif sunny == 0:
        matrix[0] = [0.25, 0.25, 0.25, 0.25]
    if cloudy > 0:
        matrix[1] = matrix[1] / cloudy
    elif cloudy == 0:
        matrix[1] = [0.25, 0.25, 0.25, 0.25]
    if rainy > 0:
        matrix[2] = matrix[2] / rainy
    elif rainy == 0:
        matrix[2] = [0.25, 0.25, 0.25, 0.25]
    if snowy > 0:
        matrix[3] = matrix[3] / snowy
    elif snowy == 0:
        matrix[3] = [0.25, 0.25, 0.25, 0.25]
    return matrix

In [14]:
t1 = preprocessing(table_np1)
t2 = preprocessing(table_np2)
t3 = preprocessing(table_np3)
t4 = preprocessing(table_np4)
t5 = preprocessing(table_np5)
t6 = preprocessing(table_np6)
t7 = preprocessing(table_np7)
t8 = preprocessing(table_np8)
t9 = preprocessing(table_np9)
t10 = preprocessing(table_np10)
t11 = preprocessing(table_np11)
t12 = preprocessing(table_np12)

In [15]:
M1 = func_coef(t1)
M2 = func_coef(t2)
M3 = func_coef(t3)
M4 = func_coef(t4)
M5 = func_coef(t5)
M6 = func_coef(t6)
M7 = func_coef(t7)
M8 = func_coef(t8)
M9 = func_coef(t9)
M10 = func_coef(t10)
M11 = func_coef(t11)
M12 = func_coef(t12)

In [16]:
M1

array([[0.26984127, 0.46031746, 0.03174603, 0.23809524],
       [0.08196721, 0.54098361, 0.01639344, 0.36065574],
       [0.        , 0.23076923, 0.15384615, 0.61538462],
       [0.0495283 , 0.25707547, 0.00943396, 0.68396226]])

In [17]:
M = np.array([])
M = np.append(M, M1)
M = np.append(M, M2)
M = np.append(M, M3)
M = np.append(M, M4)
M = np.append(M, M5)
M = np.append(M, M6)
M = np.append(M, M7)
M = np.append(M, M8)
M = np.append(M, M9)
M = np.append(M, M10)
M = np.append(M, M11)
M = np.append(M, M12)

In [18]:
M = np.reshape(M, (12,4,4))

In [19]:
np.save('matrices/matrix1', M)  

In [20]:
my_matrix = np.load(open("matrices/matrix1.npy","rb"))

In [21]:
my_matrix

array([[[0.26984127, 0.46031746, 0.03174603, 0.23809524],
        [0.08196721, 0.54098361, 0.01639344, 0.36065574],
        [0.        , 0.23076923, 0.15384615, 0.61538462],
        [0.0495283 , 0.25707547, 0.00943396, 0.68396226]],

       [[0.24637681, 0.50724638, 0.01449275, 0.23188406],
        [0.09797297, 0.51013514, 0.02027027, 0.37162162],
        [0.06666667, 0.4       , 0.2       , 0.33333333],
        [0.06232295, 0.29745042, 0.01416431, 0.62606232]],

       [[0.25663717, 0.48672566, 0.04424779, 0.21238938],
        [0.15015974, 0.514377  , 0.07348243, 0.26198083],
        [0.05555556, 0.38888889, 0.27777778, 0.27777778],
        [0.11409396, 0.25838926, 0.03691275, 0.59060403]],

       [[0.48214286, 0.35714286, 0.11160714, 0.04910714],
        [0.25      , 0.44520548, 0.21575342, 0.0890411 ],
        [0.1625    , 0.3375    , 0.3875    , 0.1125    ],
        [0.16504854, 0.27184466, 0.10679612, 0.45631068]],

       [[0.58419244, 0.2371134 , 0.17869416, 0.        ],
      

$\begin{equation*}
T =
\begin{pmatrix}
su\rightarrow su & su\rightarrow c  & su\rightarrow r & su\rightarrow sn\\
c\rightarrow su & c\rightarrow c & c\rightarrow r & c\rightarrow sn\\
r\rightarrow su & r\rightarrow c & r\rightarrow r & r\rightarrow sn\\
sn\rightarrow su & sn\rightarrow c & sn\rightarrow r & sn\rightarrow sn
\end{pmatrix}
\end{equation*}$

In [22]:
def temp_comp(data):
    result = np.array([])
    i = 0
    while i < data.shape[0]-1:
        if data[i+1][1] - data[i][1] < -5:
            result = np.append(result, 0)
        elif (data[i+1][1] - data[i][1] >= -5) and (data[i+1][1] - data[i][1] < -2):
            result = np.append(result, 1)
        elif (data[i+1][1] - data[i][1] >= -2) and (data[i+1][1] - data[i][1] <= 2):
            result = np.append(result, 2)
        elif (data[i+1][1] - data[i][1] > 2) and (data[i+1][1] - data[i][1] <= 5):
            result = np.append(result, 3)
        else:
            result = np.append(result, 4)
        i += 1
    return result

In [23]:
diff1 = temp_comp(t1)
diff2 = temp_comp(t2)
diff3 = temp_comp(t3)
diff4 = temp_comp(t4)
diff5 = temp_comp(t5)
diff6 = temp_comp(t6)
diff7 = temp_comp(t7)
diff8 = temp_comp(t8)
diff9 = temp_comp(t9)
diff10 = temp_comp(t10)
diff11 = temp_comp(t11)
diff12 = temp_comp(t12)

In [24]:
def temp_matrix(data):
    l5 = 0
    l2 = 0
    b = 0
    m2 = 0
    m5 = 0
    matrix = np.zeros((5, 5))
    i = 0
    while i < data.shape[0]-1:
        if data[i] == 0:
            l5 += 1
            matrix[0][int(data[i+1])] += 1
        elif data[i] == 1:
            l2 += 1
            matrix[1][int(data[i+1])] += 1
        elif data[i] == 2:
            b += 1
            matrix[2][int(data[i+1])] += 1
        elif data[i] == 3:
            m2 += 1
            matrix[3][int(data[i+1])] += 1
        else:
            m5 += 1
            matrix[4][int(data[i+1])] += 1
        i += 1
    if l5 > 0:
        matrix[0] = matrix[0] / l5
    elif l5 == 0:
        matrix[0] = [0.2, 0.2, 0.2, 0.2, 0.2]
    if l2 > 0:
        matrix[1] = matrix[1] / l2
    elif l2 == 0:
        matrix[1] = [0.2, 0.2, 0.2, 0.2, 0.2]
    if b > 0:
        matrix[2] = matrix[2] / b
    elif b == 0:
        matrix[2] = [0.2, 0.2, 0.2, 0.2, 0.2]
    if m2 > 0:
        matrix[3] = matrix[3] / m2
    elif b == 0:
        matrix[3] = [0.2, 0.2, 0.2, 0.2, 0.2]
    if m5 > 0:
        matrix[4] = matrix[4] / m5
    elif m5 == 0:
        matrix[4] = [0.2, 0.2, 0.2, 0.2, 0.2]
    return matrix

In [25]:
T1 = temp_matrix(diff1)
T2 = temp_matrix(diff2)
T3 = temp_matrix(diff3)
T4 = temp_matrix(diff4)
T5 = temp_matrix(diff5)
T6 = temp_matrix(diff6)
T7 = temp_matrix(diff7)
T8 = temp_matrix(diff8)
T9 = temp_matrix(diff9)
T10 = temp_matrix(diff10)
T11 = temp_matrix(diff11)
T12 = temp_matrix(diff12)

In [26]:
T1

array([[0.02597403, 0.03896104, 0.20779221, 0.2987013 , 0.42857143],
       [0.04411765, 0.08823529, 0.33823529, 0.33088235, 0.19852941],
       [0.08108108, 0.15945946, 0.57567568, 0.13783784, 0.04594595],
       [0.13768116, 0.27536232, 0.49275362, 0.07246377, 0.02173913],
       [0.24096386, 0.28915663, 0.3373494 , 0.09638554, 0.03614458]])

In [27]:
T = np.array([])
T = np.append(T, T1)
T = np.append(T, T2)
T = np.append(T, T3)
T = np.append(T, T4)
T = np.append(T, T5)
T = np.append(T, T6)
T = np.append(T, T7)
T = np.append(T, T8)
T = np.append(T, T9)
T = np.append(T, T10)
T = np.append(T, T11)
T = np.append(T, T12)

In [28]:
T = np.reshape(T, (12,5,5))

In [29]:
np.save('matrices/matrix2', T)  

In [30]:
my_matrix2 = np.load(open("matrices/matrix2.npy","rb"))

In [31]:
my_matrix2

array([[[0.02597403, 0.03896104, 0.20779221, 0.2987013 , 0.42857143],
        [0.04411765, 0.08823529, 0.33823529, 0.33088235, 0.19852941],
        [0.08108108, 0.15945946, 0.57567568, 0.13783784, 0.04594595],
        [0.13768116, 0.27536232, 0.49275362, 0.07246377, 0.02173913],
        [0.24096386, 0.28915663, 0.3373494 , 0.09638554, 0.03614458]],

       [[0.        , 0.02531646, 0.10126582, 0.34177215, 0.53164557],
        [0.0141844 , 0.04255319, 0.29787234, 0.5106383 , 0.13475177],
        [0.09540636, 0.20141343, 0.49116608, 0.16254417, 0.04946996],
        [0.18421053, 0.375     , 0.40131579, 0.02631579, 0.01315789],
        [0.28571429, 0.24675325, 0.42857143, 0.02597403, 0.01298701]],

       [[0.        , 0.04      , 0.1       , 0.23      , 0.63      ],
        [0.01156069, 0.        , 0.19075145, 0.5433526 , 0.25433526],
        [0.0995671 , 0.22077922, 0.47619048, 0.16017316, 0.04329004],
        [0.23717949, 0.46153846, 0.29487179, 0.00641026, 0.        ],
        [0.32478

In [32]:
test = np.array([0.3, 0.7, 0, 0]) @ M5 

In [33]:
list1 = ['Ясно', 'Облачно', 'Дождь', 'Снег',]

In [34]:
from functools import cmp_to_key

def my_cmp(a, b):
    return b[0] - a[0]

def translation1(vector):
    maximum = max(vector)
    res1 = []
    res2 = []
    vector = vector / maximum
    support = [(vector[i], int(i)) for i in range(len(vector))]
    support = sorted(support, key=cmp_to_key(my_cmp))
    print(support)
    names = list1[:4]
    i = 0
    while i < len(vector):
        if support[i][0] > 0.8:
            res1.append(names[support[i][1]])
        elif support[i][0] >= 0.5:
            res2.append(names[support[i][1]])
        i += 1
    return res1, res2

In [35]:
d1, d2 = translation1(test)

[(1.0, 0), (0.9237920196047094, 1), (0.8834771722337648, 2), (0.009043204943796282, 3)]


In [36]:
test

array([0.35507425, 0.32801475, 0.31369999, 0.00321101])

In [37]:
name = ''
for i in d1+d2:
    name += i + ', '

In [38]:
name

'Ясно, Облачно, Дождь, '